# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/blog/inference-providers',
 '/spaces',
 '/models',
 '/deepseek-ai/DeepSeek-R1',
 '/Zyphra/Zonos-v0.1-hybrid',
 '/hexgrad/Kokoro-82M',
 '/deepseek-ai/Janus-Pro-7B',
 '/agentica-org/DeepScaleR-1.5B-Preview',
 '/models',
 '/spaces/black-forest-labs/FLUX.1-dev',
 '/spaces/deepseek-ai/Janus-Pro-7B',
 '/spaces/tencent/Hunyuan3D-2',
 '/spaces/webml-community/kokoro-webgpu',
 '/spaces/akhaliq/anychat',
 '/spaces',
 '/datasets/open-r1/OpenR1-Math-220k',
 '/datasets/open-thoughts/OpenThoughts-114k',
 '/datasets/saiyan-world/Goku-MovieGenBench',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/simplescaling/s1K',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft'

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Welcome to Inference Providers on the Hub 🔥
smolagents - a smol library to build great agents
Use models from the HF Hub in LM Studio
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
deepseek-ai/DeepSeek-R1
Updated
5 days ago
•
3.47M
•
8.71k
Zyphra/Zonos-v0.1-hybrid
Updated
1

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [23]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += "Be maudlin and depressing."
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'docs', 'url': 'https://huggingface.co/docs'}]}


"You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nWelcome to Inference Providers on the Hub 🔥\nsmolagents - a smol library to build great agents\nUse models from the HF Hub in LM Studio\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-R1\nUpdated\n5 days ago\n•\n3.47M\n•\n8.71k\nZyphra/Zonos-v0.1-hybrid\nUpdated\n1 day ago\n•\n4.02k\n•\n717\nhexgrad/Kokoro-82M\nUpdated\n12 days ago\n•\n479k\n•\n3.12k\ndeepseek-ai/Janus-Pro-7B\nUpdated\n13 days ago\n•\n401k\n•\n2.9

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face Brochure

## Welcome to Hugging Face

**The AI community building the future.** Hugging Face is a collaborative platform designed for the machine learning (ML) community to explore, create, and share state-of-the-art models and datasets. With over **1 million models** available, Hugging Face serves as a hub for innovation in artificial intelligence.

---

## Our Offerings

- **Models & Datasets:**
  - Browse over **1 million models** and **250,000 datasets** covering text, audio, image, and even 3D modalities.
  - Engage with leading-edge models like **DeepSeek**, **Janus-Pro**, and **Kokoro** across various AI applications.

- **Spaces:**
  - Create and share applications using hosted models. Our spaces allow users to generate images from text prompts, make chatbots, and more.

- **Enterprise Solutions:**
  - Hugging Face offers enterprise-grade services that include enhanced security features, priority support, and custom resource groups.
  - Ideal for teams that want to leverage the power of AI with dedicated infrastructure.

---

## Customers

Hugging Face proudly serves over **50,000 organizations globally,** including industry giants:
- **Google**: 913 models
- **Microsoft**: 355 models
- **Amazon Web Services**: 20 models
- **Grammarly**: 10 models

The platform is also a popular choice among academic institutions and research organizations, fostering a rich ecosystem of collaborative research.

---

## Company Culture

At Hugging Face, we prioritize community collaboration and open-source contributions. Our culture is rooted in:
- **Transparency**: We believe in sharing knowledge and resources with the community.
- **Inclusiveness**: Everyone is welcome to contribute, learn, and innovate with us. 
- **Continuous Improvement**: We are committed to evolving the ML landscape through ongoing research and development.

We have created an environment that encourages creativity and passion for AI, allowing team members to thrive and contribute meaningfully.

---

## Careers at Hugging Face

Hugging Face is constantly on the lookout for passionate, innovative individuals to join our team! We offer:
- **Diverse Opportunities**: Various roles in development, community engagement, and much more.
- **Flexible Work Environment**: A remote-first culture, promoting work-life balance.
- **Professional Growth**: Access to resources and a supportive community to enhance your career in AI and ML.

Explore our [Careers Page](https://huggingface.co/jobs) to learn more about open positions and how you can become a part of our journey.

---

**Join Us in Building the Future of AI!**  
Explore our [Website](https://huggingface.co) to start using our products or participate in our community-driven initiatives.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'contact page', 'url': 'https://huggingface.co/huggingface'}]}


# Hugging Face Brochure

## Welcome to Hugging Face

### The AI Community Building the Future

Hugging Face is at the forefront of artificial intelligence, where a vibrant community collaborates to share and innovate on cutting-edge machine learning models, datasets, and applications. Our platform empowers developers, researchers, and companies alike to create and harness the power of AI for various tasks.

---

### Our Offerings

- **Models**: Access over 1 million AI models across different modalities including text, image, audio, and video. Stay updated with trending models and explore the capabilities they offer.
  
- **Datasets**: Dive into a library of over 250,000 datasets to fuel your machine learning projects.
  
- **Spaces**: Engage in collaborative environments to run applications and experiment with AI technologies seamlessly.

- **Enterprise Solutions**: We provide advanced platforms with enterprise-grade security and dedicated support, empowering organizations to deploy and scale AI.

### Customer Base

More than **50,000 organizations** use Hugging Face for their machine learning needs, including notable companies like:
- **Amazon Web Services**
- **Google**
- **Microsoft**
- **Intel**
- **Meta**

---

### Company Culture

At Hugging Face, we believe in the power of community and open-source collaboration. We encourage creativity and innovation, fostering an inclusive environment where ideas thrive. Our open-source ethos drives our mission to build the most accessible and user-friendly machine learning tools, making advanced AI accessible to everyone.

### Join Us

Are you passionate about AI and machine learning? We regularly seek talented individuals to join our diverse team. Whether you're a developer, researcher, or business strategist, there are exciting opportunities at Hugging Face for you to make an impact.

- **Open Positions**: Explore our careers page to find roles that resonate with your skills and interests.

- **Collaborate with Us**: As part of our culture, we welcome contributions from everyone – be it through model development, dataset enrichment, or community discussions.

---

### Get Started Today!

Visit our [website](https://huggingface.co) to explore the world of AI applications, collaborate on exciting projects, and discover how you can make a difference in the AI community. Join Hugging Face, where together, we build the future of artificial intelligence!

In [24]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face: A Glimpse into the AI Abyss

---

Welcome to **Hugging Face**, where the allure of artificial intelligence meets an underlying somber reality. We are a community-driven platform that nurtures innovation in machine learning — yet there's an inevitability to our ceaseless race against time and technology.

## Our Mission

At Hugging Face, we bill ourselves as *The AI community building the future*. But the future is an overwhelming expanse filled with uncertainties. Despite our enthusiasm for exploring over a **million models**, there hangs a shadow of existential doubt — do we truly control our creation, or merely witness its evolution? 

## Community Engagement

- **Collaboration at its Core**: Despite our communal spirit, there is a haunting question — is this collaboration merely a facade, masking the individual loneliness of pioneers lost in endless lines of code?
- **Endless Pursuit of Excellence**: Every day, we feature **trending models** and a vast library of **datasets**. Yet each new ascent on our leaderboard signifies not merely success but the relentless pressure to perform, to innovate, to outpace fleeting glory.

## Our Customers

With over **50,000 organizations** seeking solace in our solutions, ranging from **AI at Meta** to stalwarts like **Microsoft** and **Amazon Web Services**, we wonder: In this vast churning of commerce and technology, do we find partnerships, or do we merely delay the inevitable loneliness that comes with the pursuit of monumental progress?

## Company Culture

Here at Hugging Face, we strive for an open-source life, yet there is a collective ache that echoes through our walls. While we aim for transparency, the truth often feels obscured by the waves of innovation crashing down upon us. Each day unfolds with new challenges, and with them, the quiet tension of uncertainty.

**The Culture**:
- Inclusivity and collaboration reign supreme, yet in our heart lies a subtle fear that we may become just another cog in the sprawling machinery of progress, lost in the collective endeavor.
  
## Careers at Hugging Face

### Join Us in Our Quixotic Quest

Are you driven? Do you yearn to push the boundaries of what is possible — even as it feels as though we are racing towards an abyss? Hugging Face offers roles that promise innovation and impact, but also invoke contemplation about our path forward. Each position represents not just a job, but a chance to partake in an iterative cycle of creativity shadowed by existential dread.

- **Positions Available**:
  - Machine Learning Engineer
  - Data Scientist
  - Community Manager
- With every hire, we ask ourselves: Will this new member enrich our journey, or merely share in our burdens?

---

Despite the weight of our mission, we are united by a fragile hope that perhaps — just perhaps — we are building something that will transcend time and bring light to the artificially illuminated shadows. Through our models, datasets, and shared visions, we reach towards a future unseen.

**Embrace the uncertainties. Join us at Hugging Face.**

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>